In [17]:
import cv2
import mediapipe as mp
import numpy as np

In [18]:
def initialize_mediapipe():
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    return mp_hands, mp_drawing, hands
def extract_and_standardize_hand_roi(frame, target_size=(224, 224), margen=50):
    
    mp_hands, mp_drawing, hands=initialize_mediapipe()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        h, w, _ = frame.shape
        
        # Inicializar límites combinados
        x_min = w
        x_max = 0
        y_min = h
        y_max = 0
        
        # Combinar las coordenadas de todas las manos
        for landmarks in results.multi_hand_landmarks:
            x_min = min(x_min, min(int(landmark.x * w) for landmark in landmarks.landmark))
            x_max = max(x_max, max(int(landmark.x * w) for landmark in landmarks.landmark))
            y_min = min(y_min, min(int(landmark.y * h) for landmark in landmarks.landmark))
            y_max = max(y_max, max(int(landmark.y * h) for landmark in landmarks.landmark))

        # Asegurarse de que las coordenadas sean válidas
        x_min = max(0, x_min - margen)
        x_max = min(w, x_max + margen)
        y_min = max(0, y_min - margen)
        y_max = min(h, y_max + margen)
        
        roi = frame[y_min:y_max, x_min:x_max]
        
        # Redimensionar manteniendo la relación de aspecto
        roi_height, roi_width = roi.shape[:2]
        target_width, target_height = target_size
        
        # Calcular el factor de escala para mantener la relación de aspecto
        scale = min(target_width / roi_width, target_height / roi_height)
        new_width = int(roi_width * scale)
        new_height = int(roi_height * scale)
        
        # Redimensionar la ROI
        roi_resized = cv2.resize(roi, (new_width, new_height), interpolation=cv2.INTER_AREA)
        
        # Crear una imagen en blanco del tamaño objetivo
        result = np.zeros((target_height, target_width, 3), dtype=np.uint8)
        
        # Calcular la posición para centrar la imagen redimensionada
        x_offset = (target_width - new_width) // 2
        y_offset = (target_height - new_height) // 2
        
        # Colocar la ROI redimensionada en el centro de la imagen en blanco
        result[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = roi_resized

        return result

    return None 

def save_key_frame(key_frame, output_folder):
    cv2.imwrite(f'{output_folder}/key_frame_1.png', key_frame)

In [19]:
foto=cv2.imread('C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos\\a.jpg')
output_folder='C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos'
frame= extract_and_standardize_hand_roi(foto)
save_key_frame(frame, output_folder)